# Résolution Pb65 de la collection Hock-Schittkowski

Classification dans l'ouvrage : QQR-P1-3

Number of variables : n = 3

Number of constraints : 7 (1 inequality + 6 bounds)

# Modèle

$\left\{  
\begin{array}{lllll} 
\min f(x) \\ 
\text{s.c.}\\
 48-x_1^2-x_2^2-x_3^2\geq 0\\
-4.5\leq x_i\leq 4.5,\text{ pour } i=1,2\\
-5 \leq x_3 \leq 5
\end{array} \right.$

où $f : x \longmapsto (x_1-x_2)^2 + \dfrac{(x_1+x_2-10)^2}{9}+(x_3-5)^2=||r(x)||^2$

avec $r : x\longmapsto  (x_1-x_2,\dfrac{x_1-x_2-10}{3},x_3-5)$

Point de départ : $x_0 = (-5,5,0)$ (non réalisable)

Fonction objectif en $x_0$ : $\dfrac{1225}{9}$

Solution attendue : $x^* = (3.650461821,3.65046168,4.6204170507)$

Fonction objectif à la solution : $f(x^*) = 0.935288567$

In [ ]:
using BenchmarkTools

In [ ]:
include("../src/enlsip_functions.jl")

In [ ]:
n = 3
m = 3
nb_eq = 0
nb_constraints = 7

In [ ]:
res65 = ResidualsEval(0)

function (res65::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(res65.ctrl) == 1
        rx[:] = [x[1] - x[2]; (x[1]+x[2]-10.0) / 3.0; x[3]-5.0]

    # The jacobian is computed analytically
    elseif res65.ctrl == 2
        J[:] = [1. -1. 0;
                1/3 1/3 0.;
                0. 0. 1.]
    end
    return
end

In [ ]:
cons65 = ConstraintsEval(0)

function (cons65::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(cons65.ctrl) == 1
        cx[:] = [48.0 - x[1]^2-x[2]^2-x[3]^2;
                 x[1]+4.5;
                 x[2]+4.5;
                 x[3]+5.0;
                 -x[1]+4.5;
                 -x[2]+4.5;
                 -x[3]+5.0]
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif cons65.ctrl == 2
        cons65.ctrl = 0
    end
    return
end

In [ ]:
x0 = [-5.0;5.0;0.0]
e = eps(Float64)
se = sqrt(e)
enlsip65 = enlsip(x0,res65,cons65,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)

In [ ]:
x1_sol = enlsip65.sol[1]
x2_sol = enlsip65.sol[2]
x3_sol = enlsip65.sol[3]

println("Solution trouvée avec ENLSIP-Julia :")
@printf "x_jul = (%.9e, %.9e, %.9e)\n" x1_sol x2_sol x3_sol
@printf "f(x_jul) = %.9e\n\n" enlsip65.obj_value  

println("Solution trouvée avec ENLSIP-Fortran :")
@printf "x_for = (%.7e, %.7e, %.7e)\n"  3.6504617  3.6504617  4.6204176
@printf "f(x_for) = %.5e\n"  0.953529

println("\nValeurs théoriques visées :")
@printf "x_opt = (%.9e, %.9e, %.9e)\n" 3.650461821 3.65046168 4.6204170507
@printf "f(x_opt) = %.9e" 0.9535288567

# Résolution avec Ipopt

In [ ]:
using JuMP, Ipopt

In [ ]:
model = Model(with_optimizer(Ipopt.Optimizer))

@variable(model, x1, start = -5.0)
@variable(model, x2, start = 5.0)
@variable(model, x3, start = 0.0)

In [ ]:
f(x1,x2,x3) = (x1 - x2)^2 + (x1+x2-10.0)^2 / 9.0 + (x3-5.0)^2
JuMP.register(model, :f, 3, f, autodiff=true)

In [ ]:
@NLconstraint(model, c1, 48.0 - x1^2 - x2^2 - x3^2 >= 0)
@constraint(model, x1 >= -4.5)
@constraint(model, -x1 <= 4.5)
@constraint(model, x2 >= -4.5)
@constraint(model, x2 <= 4.5)
@constraint(model, x3 >= -5.0)
@constraint(model, x3 <= 5.0)

In [ ]:
@NLobjective(model, Min, f(x1,x2,x3))

In [ ]:
JuMP.optimize!(model)

In [ ]:
println("Solution trouvée avec ENLSIP :")
@printf "x_enlsip = (%.9e, %.9e, %.9e)\n" x1_sol x2_sol x3_sol
@printf "f(x_enlsip) = %.9e\n\n" enlsip65.obj_value 


println("Solution trouvée avec Ipopt :")
@printf "x_ipopt = (%.9e, %.9e, %.9e)\n" JuMP.value(x1) JuMP.value(x2) JuMP.value(x3)
@printf "f(x_ipopt) = %.9e\n" 0.95352885599324944